In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [21]:
base_url = 'https://www.presidency.ucsb.edu'
# to match Liu and Lei, dates should be from "April 1, 2015 for Clinton and June 16th, 2015 for Trump"
# to Trump’s victory and Clinton’s concession speeches on November 9, 2016
init_url = 'https://www.presidency.ucsb.edu/documents/app-categories/elections-and-transitions/campaign-documents?items_per_page=250&field_docs_start_date_time_value%5Bvalue%5D%5Bdate%5D=2016'

In [22]:
r = requests.get(init_url)
s = BeautifulSoup(r.text, 'html.parser')

In [23]:
dates = [d.find("span",attrs={'class':'date-display-single'})['content'] for d in s.find_all("div",attrs={'class':'views-row'})]
doc_links = [d.find("div",attrs={'class':'field-title'}).find("a") for d in s.find_all("div",attrs={'class':'views-row'})]
titles = [d.string for d in doc_links]
links = [d['href'] for d in doc_links]
persons = [d.find("div",attrs={'class':'col-sm-4'}).find("a").string for d in s.find_all("div",attrs={'class':'views-row'})]

In [27]:
sdf = pd.DataFrame({"title": titles, "date": dates, "person": persons, "link": links})

In [40]:
sdf["transcript"] = ""

In [48]:
# This will take a few minutes to download them all.
sdf = sdf.set_index('link')
for link in links:
    #link = '/documents/remarks-new-york-city-accepting-election-the-45th-president-the-united-states'
    #fname = link[11:]
    r = requests.get(base_url + link)
    s = BeautifulSoup(r.text, 'html.parser')
    transcript = s.find("div",attrs={'class':'field-docs-content'}).get_text()
    sdf.at[link, "transcript"] = transcript
    
sdf = sdf.reset_index()

In [50]:
# at this point it includes speeches by other people - Obama, Sanders
# it also includes dates outside of the campaign season
#sdf.to_json("data/ucsb_speeches_2016_all.json")

In [60]:
# this cuts the number of transcripts about in half
sdf = sdf[sdf['person'].isin(['Donald J. Trump','Hillary Clinton']) & sdf['date'].between('2016-07-28','2016-11-08')]

In [61]:
sdf.to_json("data/ucsb_speeches_2016.json")

In [62]:
sdf

,link,title,date,person,transcript
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07T00:00:00+00:00,Hillary Clinton,\nHello Grand Valley! Thank you! Thank you.\nI...
3,/documents/remarks-js-dorton-arena-raleigh-nor...,"Remarks at J.S Dorton Arena in Raleigh, North ...",2016-11-07T00:00:00+00:00,Donald J. Trump,\nThis is amazing. What a great place. Thank y...
4,/documents/remarks-north-carolina-state-univer...,Remarks at North Carolina State University in ...,2016-11-07T00:00:00+00:00,Hillary Clinton,\nThank you all! Whoa! Thank you! Thank you. W...
5,/documents/remarks-the-university-pittsburgh,Remarks at the University of Pittsburgh,2016-11-07T00:00:00+00:00,Hillary Clinton,"\nHello, Pittsburgh! Woah! Hello back there!\n..."
6,/documents/remarks-eastern-market-detroit-mich...,"Remarks at Eastern Market in Detroit, Michigan",2016-11-04T00:00:00+00:00,Hillary Clinton,"\nHello Detroit! Thank you! I'll tell you, I a..."
...,...,...,...,...,...
103,/documents/remarks-erie-insurance-arena-erie-p...,"Remarks at Erie Insurance Arena in Erie, Penns...",2016-08-12T00:00:00+00:00,Donald J. Trump,"\nThank you. Thank you, everybody. Thank you. ..."
104,/documents/remarks-rally-the-university-north-...,Remarks at a Rally at the University of North ...,2016-08-09T00:00:00+00:00,Donald J. Trump,\nThank you. Thank you very much. [applause] T...
105,/documents/remarks-the-detroit-economic-club-1,Remarks to the Detroit Economic Club,2016-08-08T00:00:00+00:00,Donald J. Trump,\nThank you for the invitation to speak to you...
106,/documents/remarks-the-ki-convention-center-gr...,Remarks at the KI Convention Center in Green B...,2016-08-05T00:00:00+00:00,Donald J. Trump,\nSo how good is he? How good is he? [applause...


In [1]:
#Liu and Lei: 
"""Specifically, the Clinton corpus contains 89 scripts with a total of 286,899 words while the
Trump corpus includes 74 scripts with a total of 276,212 words."""

'Specifically, the Clinton cor-\npus contains 89 scripts with a total of 286,899 words while the\nTrump corpus includes 74 scripts with a total of 276,212 words.'